In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
import requests

In [2]:
!pip install folium==0.10.1

     |████████████████████████████████| 92kB 14.2MB/s eta 0:00:01


In [3]:
import folium

In [4]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
4,4,North West Delhi,Begum Pur,NaN,NaN


In [5]:
df=df_data_0
df = df.dropna()
latitude=28.6139
longitude=77.2090

In [6]:
map=folium.Map(location=[28.6139, 77.2090],tiles="stamen terrain")

In [7]:
for lat, lon in zip(df['latitude'],df['longitude']):
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map)
map

In [8]:
CLIENT_ID = 'RX0UN14OAIVSBX5V1K5ZK1UAIDPOLBZ24SJK3FO3ZY05VF3S' # your Foursquare ID
CLIENT_SECRET = '1WJXHBFL1ED4ZWSYOLSN2EENMV1YFQSRXPZFTQO5V53KOEIH' # your Foursquare Secret
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RX0UN14OAIVSBX5V1K5ZK1UAIDPOLBZ24SJK3FO3ZY05VF3S
CLIENT_SECRET:1WJXHBFL1ED4ZWSYOLSN2EENMV1YFQSRXPZFTQO5V53KOEIH


In [9]:
k=5
dfclustering = df.drop(['Unnamed: 0','Neighborhood','Borough'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(dfclustering)
kmeans.labels_
df.insert(0, 'Cluster Labels', kmeans.labels_)

In [10]:
# Combining the neighbourhoods with same Postalcode
df2 = df.groupby(['Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)
df2=df2.join(df['longitude'])
df2=df2.join(df['latitude'])
df2.drop(4,axis=0)
df2

,Borough,Neighborhood,longitude,latitude
0,North West Delhi,"Adarsh Nagar, Ashok Vihar, Azadpur, Bawana, Dh...",77.071541,28.614192
1,North Delhi,"Shakti Nagar, Bara Hindu Rao, Chandni Chowk, C...",77.184826,28.699453
2,North East Delhi,"Babarpur, Dilshad Garden, Kabir Nagar, Mandoli...",77.175547,28.707657
3,Central Delhi,"Chandni Chowk, Daryaganj, Jhandewalan, Karol B...",77.032885,28.799660
4,New Delhi,"Barakhamba Road, Chanakyapuri, Connaught Place...",NaN,NaN
5,East Delhi,"Brij Puri, East Vinod Nagar, Gandhi Nagar, Gee...",-90.261223,39.031714
6,South Delhi,"Alaknanda, Chhattarpur, Chittaranjan Park, Day...",77.162658,28.725972
7,South West Delhi,"Delhi Cantonment, Dhaula Kuan, Ghitorni, Inder...",77.032511,28.735140
8,West Delhi,"Ashok Nagar, Bali Nagar, Dhaula Kuan, Fateh Na...",77.161683,28.688926


In [11]:
map_clusters = folium.Map(location=[28.6139, 77.2090],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, borough, cluster in zip(df['latitude'], df['longitude'], df['Borough'], df['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [12]:
CLIENT_ID = 'RX0UN14OAIVSBX5V1K5ZK1UAIDPOLBZ24SJK3FO3ZY05VF3S' # your Foursquare ID
CLIENT_SECRET = '1WJXHBFL1ED4ZWSYOLSN2EENMV1YFQSRXPZFTQO5V53KOEIH' # your Foursquare Secret
version=20200305
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10500 # define radius
latitude=28.6139
longitude=77.2090
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    version, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=RX0UN14OAIVSBX5V1K5ZK1UAIDPOLBZ24SJK3FO3ZY05VF3S&client_secret=1WJXHBFL1ED4ZWSYOLSN2EENMV1YFQSRXPZFTQO5V53KOEIH&v=20200305&ll=28.6139,77.209&radius=10500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9f18ee78a484001b364806'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Delhi',
  'headerFullLocation': 'Delhi',
  'headerLocationGranularity': 'city',
  'totalResults': 233,
  'suggestedBounds': {'ne': {'lat': 28.708400094500096,
    'lng': 77.316446425829},
   'sw': {'lat': 28.519399905499906, 'lng': 77.101553574171}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54dc85c7498ef8f9ab9b3c08',
       'name': 'Tamra',
       'location': {'address': '19, Ashoka Rd',
        'crossStreet': 'Windsor Pl',
        'lat': 28.620543184230115,
        'lng': 77.21817445898111,
        'labeledLatLngs': [{'label': 'display',
          'lat': 28.62054

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues

#column formatting
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,id,categories,lat,lng
0,Tamra,54dc85c7498ef8f9ab9b3c08,Restaurant,28.620543,77.218174
1,The Imperial,4bc8f2c7762beee1a8bb3d38,Hotel,28.625548,77.218664
2,Pandey Paan,4f8c4443e4b0a95f9685c9f6,Smoke Shop,28.622249,77.201075
3,Varq | वर्क,4c65834faebea5939e8870d0,Indian Restaurant,28.604547,77.223781
4,Gurudwara Sri Rakabganj Sahibji,4f05bbb077c8f87cd1afe77b,Spiritual Center,28.618296,77.205269
5,The Big Chill Cafe,4b701cbaf964a5209b072de3,Italian Restaurant,28.600686,77.227636
6,The Claridges,4bd728fb304fce72582833ab,Hotel,28.600705,77.216601
7,Spa At Shangri-La,4f8c11ece4b0af4d504bbac1,Spa,28.620869,77.218289
8,Naturals Ice Cream,54783eab498e910f8bd1781d,Ice Cream Shop,28.634455,77.222139
9,Amar Jawan Jyoti | अमर जवान ज्योति (Amar Jawan...,4f215396e4b028522e2f3745,Sculpture Garden,28.612980,77.228247


In [16]:
nearby_venues['categories'].unique()

array(['Restaurant', 'Hotel', 'Smoke Shop', 'Indian Restaurant',
       'Spiritual Center', 'Italian Restaurant', 'Spa', 'Ice Cream Shop',
       'Sculpture Garden', 'Market', 'Plaza', 'South Indian Restaurant',
       'Park', 'Irani Cafe', 'Café', 'Mediterranean Restaurant', 'Bakery',
       'Coffee Shop', 'Monument / Landmark', 'French Restaurant',
       'Golf Course', 'Food Truck', 'Art Museum', 'Art Gallery',
       'North Indian Restaurant', 'Molecular Gastronomy Restaurant',
       'Theater', 'Bar', 'Clothing Store', 'Karnataka Restaurant',
       'Arcade', 'Cocktail Bar', 'Bistro', 'BBQ Joint', 'Lounge',
       'Deli / Bodega', 'Tibetan Restaurant',
       'Northeast Indian Restaurant', 'Snack Place', 'Historic Site',
       'Boutique', 'Mosque', 'Dessert Shop', 'Speakeasy', 'University'],
      dtype=object)

In [17]:
venue_id_list = nearby_venues['name'].tolist()
venue_id_list

['Tamra',
 'The Imperial',
 'Pandey Paan',
 'Varq | वर्क',
 'Gurudwara Sri Rakabganj Sahibji',
 'The Big Chill Cafe',
 'The Claridges',
 'Spa At Shangri-La',
 'Naturals Ice Cream',
 'Amar Jawan Jyoti | अमर जवान ज्योति (Amar Jawan Jyoti)',
 'Gulati Restaurant',
 'Khan Market | खान मार्केट (Khan Market)',
 'Connaught Place | कनॉट प्लेस (Connaught Place)',
 'HOTEL SARAVANA BHAVAN',
 'Lodhi Gardens (लोधी बाग़) (Lodhi Gardens)',
 'SODABOTTLEOPENERWALA',
 "L'Opera",
 'Sevilla',
 "Wenger's",
 'Perch',
 'Amour Bistro',
 'HOTEL SARAVANA BHAVAN',
 'Perch, Wine And Coffee Bar',
 'India Gate | इंडिया गेट (India Gate)',
 'Smoke House Deli',
 'Delhi Golf Club',
 'Jain Chawal Wale',
 'National Crafts Museum | राष्ट्रीय शिल्प संग्रहालय',
 'The Oberoi',
 'The Lodhi Hotel',
 'Indian Accent',
 'Bengali Market | बंगाली मार्केट | বাংলা বাজার',
 'Sagar Ratna',
 'Starbucks',
 'National Gallery Of Modern Art | राष्ट्रीय आधुनिक कला दीर्घा',
 'The Leela Palace',
 'Rajdhani Thali',
 'Bukhara',
 'ITC Maurya',
 'T

In [18]:
#set up to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in venue_id_list:
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(50, CLIENT_ID, CLIENT_SECRET,version)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)

[240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240]


In [25]:
to_venues = nearby_venues.copy()
likist=pd.DataFrame(like_list)
to_venues=to_venues.join(likist)
to_venues

,name,id,categories,lat,lng,0
0,Tamra,54dc85c7498ef8f9ab9b3c08,Restaurant,28.620543,77.218174,240
1,The Imperial,4bc8f2c7762beee1a8bb3d38,Hotel,28.625548,77.218664,240
2,Pandey Paan,4f8c4443e4b0a95f9685c9f6,Smoke Shop,28.622249,77.201075,240
3,Varq | वर्क,4c65834faebea5939e8870d0,Indian Restaurant,28.604547,77.223781,240
4,Gurudwara Sri Rakabganj Sahibji,4f05bbb077c8f87cd1afe77b,Spiritual Center,28.618296,77.205269,240
5,The Big Chill Cafe,4b701cbaf964a5209b072de3,Italian Restaurant,28.600686,77.227636,240
6,The Claridges,4bd728fb304fce72582833ab,Hotel,28.600705,77.216601,240
7,Spa At Shangri-La,4f8c11ece4b0af4d504bbac1,Spa,28.620869,77.218289,240
8,Naturals Ice Cream,54783eab498e910f8bd1781d,Ice Cream Shop,28.634455,77.222139,240
9,Amar Jawan Jyoti | अमर जवान ज्योति (Amar Jawan...,4f215396e4b028522e2f3745,Sculpture Garden,28.612980,77.228247,240


In [29]:
print(to_venues['0'].max())
print(to_venues['0'].min())
print(to_venues['0'].median())
print(to_venues['0'].mean())

KeyError: '0'

In [21]:
import matplotlib.pyplot as plt
to_venues['total likes'].hist(bins=4)
plt.show()
print(np.percentile(to_venues['total likes'], 25))
print(np.percentile(to_venues['total likes'], 50))
print(np.percentile(to_venues['total likes'], 75))

KeyError: 'total likes'

In [ ]:
poor = to_venues['total likes']<=24
below_avg = to_venues[(to_venues['total likes']>24) & (to_venues['total likes']<=45)]
abv_avg = to_venues[(to_venues['total likes']>45) & (to_venues['total likes']<=76)]
great = to_venues['total likes']>76

In [ ]:
def conditions(s):
    if s['total likes']<=24:
        return 'poor'
    if s['total likes']<=45:
        return 'below avg'
    if s['total likes']<=76:
        return 'avg avg'
    if s['total likes']>76:
        return 'great'

to_venues['total likes_cat']=to_venues.apply(conditions, axis=1)

In [ ]:
to_venues['categories'].unique()

In [ ]:


bars = ['Pub', 'Cocktail Bar', 'Bar', 'Dive Bar', 'Sports Bar']
other = ['Bagel Shop', 'Tea Room', 'Donut Shop', 'Coffee Shop', 'Bubble Tea Shop', 'Sandwich Place', 'Boutique', 'Ice Cream Shop']
euro_asia_indian_food = ['Falafel Restaurant', 'Korean Restaurant','Sushi Restaurant', 'Indian Restaurant', 'Japanese Restaurant', 'Poke Place', 'Thai Restaurant', 'Vietnamese Restaurant']
mex_southam_food = ['Cuban Restaurant', 'Mexican Restaurant', 'South American Restaurant', 'Latin American Restaurant']
american_food = ['Burger Joint', 'Restaurant', 'American Restaurant']
italian_food = ['Italian Restaurant', 'Seafood Restaurant', 'Pizza Place']

def conditions2(s):
    if s['categories'] in bars:
        return 'bars'
    if s['categories'] in other:
        return 'other'
    if s['categories'] in euro_asia_indian_food:
        return 'euro asia indian food'
    if s['categories'] in mex_southam_food:
        return 'mex southam food'
    if s['categories'] in american_food:
        return 'american food'
    if s['categories'] in italian_food:
        return 'italian food'

to_venues['categories_new']=to_venues.apply(conditions2, axis=1)

In [ ]:
to_onehot = pd.get_dummies(to_venues[['categories_new', 'total likes_cat']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Name'] = to_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

In [ ]:
cluster_df = to_onehot.drop('Name', axis=1)

k_clusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(cluster_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
to_venues['label'] = kmeans.labels_
to_venues.head()

In [ ]:
map_clusters = folium.Map(location=[28.6139, 77.2090], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_venues['lat'], to_venues['lng'], to_venues['name'], to_venues['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
to_venues.loc[to_venues['label']==0]

to_venues.loc[to_venues['label']==1]

to_venues.loc[to_venues['label']==2]

to_venues.loc[to_venues['label']==3]